# Step 2: Download and Convert the Base Model

This notebook performs the preparatory tasks needed for obtaining the base model that we will use for fine-tuning.

## Setup and Requirements
Before proceeding, you need to install one dependency to follow along. Execute the following cell before getting started.

In [1]:
! pip install huggingface-hub

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip


Please run the following cell to incorporate patches required for this hackathon material.

In [2]:
%%bash

wget -O /opt/NeMo/nemo/collections/nlp/models/language_modeling/megatron/gemma2/gemma2_modules.py \
        https://raw.githubusercontent.com/NVIDIA/NeMo/721f9101f92c9dc3976689b2bf45bad1d5075d07/nemo/collections/nlp/models/language_modeling/megatron/gemma2/gemma2_modules.py


wget -O /opt/NeMo/scripts/checkpoint_converters/convert_gemma2_hf_to_nemo.py \
        https://raw.githubusercontent.com/NVIDIA/NeMo/721f9101f92c9dc3976689b2bf45bad1d5075d07/scripts/checkpoint_converters/convert_gemma2_hf_to_nemo.py

--2024-10-28 00:39:22--  https://raw.githubusercontent.com/NVIDIA/NeMo/721f9101f92c9dc3976689b2bf45bad1d5075d07/nemo/collections/nlp/models/language_modeling/megatron/gemma2/gemma2_modules.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10842 (11K) [text/plain]
Saving to: ‘/opt/NeMo/nemo/collections/nlp/models/language_modeling/megatron/gemma2/gemma2_modules.py’

     0K ..........                                            100%  100M=0s

2024-10-28 00:39:22 (100 MB/s) - ‘/opt/NeMo/nemo/collections/nlp/models/language_modeling/megatron/gemma2/gemma2_modules.py’ saved [10842/10842]

--2024-10-28 00:39:22--  https://raw.githubusercontent.com/NVIDIA/NeMo/721f9101f92c9dc3976689b2bf45bad1d5075d07/scripts/checkpoint_converters/convert_gemma2_hf_to_nemo.py
Resol


### HuggingFace API Key

Next, please specify your HuggingFace API token. This token will be used for downloading the base model.
If you do not have an access token, follow [this tutorial](https://huggingface.co/docs/hub/en/security-tokens#how-to-manage-user-access-tokens) to generate one.

In [3]:
# Configure your HuggingFace token here.
HF_TOKEN="HF TOKEN"

## Base Model Specificaions

For this work, we will use the Gemma-2-2B model.

In [4]:
model_to_use = "google/gemma-2-2b"
model_name = model_to_use.split('/')[-1].lower()

---
## Download the Base Model from Hugging Face

Gemma 2 is a gated model on Hugging Face. To download it, you must first request access to the model by following the link [here](https://huggingface.co/google/gemma-2-2b-it)

After access has been granted, run the following cell to download the model.

In [5]:
from huggingface_hub import login, snapshot_download

# Download the model
login(token=HF_TOKEN)
snapshot_download(repo_id=model_to_use, local_dir=f"models/{model_name}")

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/818 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

'/root/ODSC-Hackathon-Repository/models/gemma-2-2b'

---
## Convert the Model to `.nemo` Format

To use the downloaded model with the NeMo Framework, we need to convert the checkpoint to the NeMo format. This is done thorugh a helper script that is already provided in the NeMo Framework container.

> NOTE: The conversion is a one-time process and may take some time to finish.

To begin the conversion process, execute the following cell.

> NOTE: If you encounter any errors during the conversion process, a log file named `model_conversion.log` will be produced in the current working directory. Please include this file when filing support requests.

In [6]:
import os
import subprocess

command = f"""
python3 /opt/NeMo/scripts/checkpoint_converters/convert_gemma2_hf_to_nemo.py \\
    --input_name_or_path ./models/{model_name} \\
    --tokenizer_path ./models/{model_name}/tokenizer.model \\
    --output_path ./models/{model_name}.nemo \\
    --run_verification
"""

# The log file to capture any messages or errors from the conversion process
log_filename = "model_conversion.log"

if os.path.exists(log_filename):
    os.remove(log_filename)

print("Model conversion started. This will take some time...")
process = subprocess.run(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

# Did the conversion process succeed?
if process.returncode == 0:
    print("Model conversion completed successfully!")
else:
    # Write the logs to a file
    with open(log_filename, "w") as f:
        f.write(process.stdout)

    print(f"Model conversion failed!")
    print(f"{'#'*80}\nLogs:\n{'#'*80}")
    print(process.stdout)
    print(f"{'#'*80}\n")
    print(f"Logs also saved to '{os.path.abspath(log_filename)}'.\nPlease share this file with us if you need help debugging the issue.")

Model conversion started. This will take some time...
Model conversion completed successfully!


---
# Freeing Memory and Other Resources

Before moving to the next notebook, please execute the following cell to free up all the allocated resources to avoid running into out-of-memory or other issues.

Alternatively, please restart the kernel by navigating to `Kernel > Restart Kernel` (if using Jypyter notebook), or clicking the `Restart` button in VS Code.

In [7]:
exit(0)